In [ ]:
%%capture
!pip uninstall --yes datasets
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip uninstall --yes fsspec
!pip install ../input/ffspecsforchallenge2/fsspec-0.9.0-py3-none-any.whl
#!pip install transformers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 
import json
import re 
from tqdm import tqdm
import tensorflow as tf
import spacy
import nltk
import torch
import os

import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input, GlobalAveragePooling1D

from collections import Counter
import transformers

import keras
tqdm.pandas()
from keras.preprocessing.sequence import pad_sequences

import argparse
import math
import random
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import numpy as np

from datasets import ClassLabel, load_dataset, load_metric

import transformers
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    HfArgumentParser,
    PreTrainedTokenizerFast,
    Trainer,
    TrainingArguments,
    set_seed,
    BertTokenizer
)
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.utils import check_min_version

In [ ]:
SUBMISSION = False #True
THRESHOLD = 0.75 #0.5
BATCH = 1536
print(BATCH)

### Load all the data

In [ ]:
test_csv = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
eval_df = pd.read_pickle("../input/give-us-the-data-in-sentences/eval_df_annotated.pkl")
train_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")

In [ ]:
eval_df = eval_df.drop(columns=['section_match', "sentence_text", "text"])
all_sets = train_df.dataset_title.unique()
eval_df = eval_df[~eval_df.label.isin(all_sets)]
eval_df.head()

A small evaluation dataframe for testing generalization

In [ ]:
eval_df = pd.concat([eval_df.loc[eval_df.sentence_match], eval_df.loc[~eval_df.sentence_match].sample(500)])

## Helping function for the processing of the data

In [ ]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def get_text(sample_id, location='/kaggle/input/coleridgeinitiative-show-us-the-data/test/'):
    """
    load the text for the given ID as string
    """
    with open(location + sample_id + ".json", "r") as file:
        sample = json.loads(file.read())

    return " ".join([s['section_title'] + " "+ s['text'] for s in sample])

def read_json_from_folder(folder_name):
    """
    load json object for the publications
    """
    
    json_dict = {}
    for filename in os.listdir(folder_name):
        with open(os.path.join(folder_name, filename)) as f:
            json_dict[filename[:-5]] = json.load(f)
    return json_dict

def get_section_data(sample_id,location='/kaggle/input/coleridgeinitiative-show-us-the-data/test/'): #train
    """
    extract the sections for each paper
    """
    with open(location + sample_id + ".json", "r") as file:
        raw_text = json.loads(file.read())
        
    return [(sample_id, s['section_title'], s['text']) for s in raw_text]

def clean_text_kaggle(txt):
    """
    the original Kaggle cleaning function
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

def custom_cleaning(text):
    """
    our custom cleaning function
    """
    # lower case text
    text = str(text).lower()
    # remove https and replace by hyperlink token -> The hyperlink itself might be a feature, by replacing it with a token we can utilize it without matching with the link text   
    text = re.sub(r'(\(?https?:[^\s]*|\(?www.[^\s]*)', "<HYPER>",  text)
    text = re.sub(f'\\n', " ", text)
    return text

def split_into_sentences(text):
    """
    tokenize with the nltk tokenizer
    """
    return tokenizer.tokenize(text)




## The Boyer Moore Matching algorithm
Retrieved from https://gist.github.com/dbrgn/1154006

In [ ]:
class last_occurrence(object):
    """Last occurrence functor."""

    def __init__(self, pattern, alphabet):
        """Generate a dictionary with the last occurrence of each alphabet
        letter inside the pattern.
        
        Note: This function uses str.rfind, which already is a pattern
        matching algorithm. There are more 'basic' ways to generate this
        dictionary."""
        self.occurrences = dict()
        for letter in alphabet:
            self.occurrences[letter] = pattern.rfind(letter)

    def __call__(self, letter):
        """Return last position of the specified letter inside the pattern.
        Return -1 if letter not found in pattern."""
        return self.occurrences[letter]


def boyer_moore_match(text, pattern):
    """Find occurrence of pattern in text."""
    alphabet = set(text)
    last = last_occurrence(pattern, alphabet)
    m = len(pattern)
    n = len(text)
    i = m - 1  # text index
    j = m - 1  # pattern index
    while i < n:
        if text[i] == pattern[j]:
            if j == 0:
                return i
            else:
                i -= 1
                j -= 1
        else:
            l = last(text[i])
            i = i + m - min(j, 1+l)
            j = m - 1 
    return -1

# Pipeline

### Preprocessing

Preprocessing the submission file for the pipeline

In [ ]:
class Submission_Prep:
    def __init__(self, csv = "../input/coleridgeinitiative-show-us-the-data/sample_submission.csv", input_folder = "../input/coleridgeinitiative-show-us-the-data/test/"):
        
        self.df = self.load_data(csv, input_folder)
        
    def load_data(self,csv, input_folder):
        df = pd.read_csv(csv)
        texts = df.Id.apply(lambda x: get_section_data(x,input_folder)).to_list()
        df = pd.DataFrame([section for pub in texts for section in pub], columns=['pub_id', 'section', 'text'])
        
        sentences = df.text.apply(split_into_sentences)
        df = df.assign(sentence_text = sentences)
        df = df.drop(columns= ["text"])
        
        # split the text into individual sentences and add as separate rows
        df =  df.explode('sentence_text')
        df = df.assign(clean_text = df.sentence_text.apply(custom_cleaning))
        return df

In [ ]:
submission = Submission_Prep().df
submission.head()

In [ ]:
TEST_INPUT_SAVE_PATH = "predict"
TEST_NER_DATA_FILE = "test_ner.json"
PREDICT_BATCH = 64000

class BertPredictor:
    
    def __init__(self,model_path:str):
        self.config = AutoConfig.from_pretrained(model_path,
                num_labels=3,
                finetuning_task='ner',
                cache_dir=None,
                revision=None,
                use_auth_token=None,
            )
        self.tokenizer = AutoTokenizer.from_pretrained(model_path,
                cache_dir=None,
                use_fast=True,
                revision=None,
                use_auth_token=None,
            )
        self.model = AutoModelForTokenClassification.from_pretrained(model_path,
                from_tf=False,
                config=self.config,
                cache_dir=False,
                revision=False,
                use_auth_token=None,
            )
        
        # create the label list with our specific entities
        self.label_list = ['B', 'I', 'O']
        self.label_to_id = {l: i for i, l in enumerate(self.label_list)}
        
        # make a folder for the predictions
        if not os.path.isdir("predict"):
            os.mkdir("predict")
        
    def predict(self, data: pd.DataFrame):
        """
        predicts and extracts the Datasets in the given data
        """
        self.write_test_data(data)
        predictions = self.get_prediction(f'{TEST_INPUT_SAVE_PATH}/{TEST_NER_DATA_FILE}')
        processed_predictions = self.tranlate_output(predictions, data)
        return processed_predictions
    
    def write_test_data(self, data: pd.DataFrame):
        """
        the prediction code expects the input as json files.
        Write one file and add dummy predictors
        """
        test_rows = [] # test data in NER format
        paper_length = [] # store the number of sentences each paper has
        sentences = data.clean_text.to_list()
        
        # collect all sentences in json
        for sentence in sentences:
            sentence_words = sentence.split()
            dummy_tags = ['O']*len(sentence_words)
            test_rows.append({'tokens' : sentence_words, 'tags' : dummy_tags})
        
        # track which sentence belongs to which data point
        paper_length.append(len(sentences))
        
        for batch_begin in range(0, len(test_rows), PREDICT_BATCH):
            # write data rows to input file
            with open(f'{TEST_INPUT_SAVE_PATH}/{TEST_NER_DATA_FILE}', 'w+') as f:
                for row in test_rows[batch_begin:batch_begin+PREDICT_BATCH]:
                    json.dump(row, f)
                    f.write('\n')

    
    def load_test_dataset(self,test_file:str):
        """
        load a Dataset from the json file
        """
        dataset =  load_dataset("json", data_files={"test": test_file})
        return dataset["test"]
    
    
    def tokenize_and_align_labels(self, examples):
        
        tokenized_inputs = self.tokenizer(
            examples[self.text_column_name],
            max_length=128,
            #padding=self.padding,
            truncation=True,
            # We use this argument because the texts in our dataset are lists of words (with a label for each word).
            is_split_into_words=True,
            )
        labels = []
        for i, label in enumerate(examples[self.label_column_name]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                # Special tokens have a word id that is None. We set the label to -100 so they are automatically
                # ignored in the loss function.
                if word_idx is None:
                    label_ids.append(-100)
                # We set the label for the first token of each word.
                elif word_idx != previous_word_idx:
                    label_ids.append(self.label_to_id[label[word_idx]])
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                else:
                    label_ids.append(self.label_to_id[label[word_idx]])
                previous_word_idx = word_idx

            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs
    
   
    def get_prediction(self,testfile:str): 
        dataset = self.load_test_dataset(testfile)
        self.column_names = dataset.column_names
        self.label_column_name = (
            "ner" if "ner" in self.column_names else self.column_names[1]
            )
        
        self.padding = "max_length"
        self.text_column_name = "tokens" if "tokens" in self.column_names else self.column_names[0]
        test_dataset = dataset.map(
                    self.tokenize_and_align_labels,
                    batched=True
                )
        
        data_collator = DataCollatorForTokenClassification(self.tokenizer)
        trainer = Trainer(
                model=self.model,
                tokenizer=self.tokenizer,
                data_collator=data_collator,
            )
        
        predictions, labels, metrics = trainer.predict(test_dataset)
        predictions = np.argmax(predictions, axis=2)
        
        # Remove ignored index (special tokens)
        true_predictions = [
            [self.label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        return true_predictions
    
    def tranlate_output(self, results, submission):
        """
        transform the output of the model back into strings. 
        """
        predicted_results = []
        for sentence, labels in zip(submission.clean_text, results):
            tokenized = self.tokenizer.tokenize(sentence)

            prediction = []
            for index, (word, label) in enumerate(zip(tokenized, labels)):
                if label == "B":
                    dataset = []
                    dataset.append(word)
                    next_id = index+1
                    while len(labels) > next_id and labels[next_id] == "I":
                        dataset.append(tokenized[next_id])
                        next_id += 1
                    prediction.append(dataset)

            # in case we get several predictions for one sample, append them
            if len(prediction) > 0:
                transformed_prediction = [self.tokenizer.convert_tokens_to_string(dataset) for dataset in prediction]
                predicted_results.append("|".join(transformed_prediction))

            # if no prediction, add empty string  
            else:
                predicted_results.append("")        
        return predicted_results
            

In [ ]:
class Pipeline:
    
    def __init__(self, lstm_path, bert_path, threshold = THRESHOLD, batch = BATCH, gpu = False):
    
        self.lstm_model = self.load_lstm(lstm_path)

        self.threshold = threshold
        self.processing_batch = batch
        
        self.bert = BertPredictor(bert_path)
        
        self.dictionary = None
        
    def predict(self,df):
        
        df = df.reset_index()
        
        print("Start String matching")
        # check for  direct string matches
        string_matched_df = self.string_matching(df)
        print("String matching done")
        
        # and split df based on the results
        string_match = string_matched_df.dropna(subset=["PredictionString"])
        string_match = string_match.assign(lstm_prediction = pd.NA)
        
        no_string_match = string_matched_df[~string_matched_df.index.isin(string_match.index)]

        # get lstm prediction
        print("Start with the LSTM predictions")
        lstm_predictions = self.lstm_prediction(no_string_match.clean_text.to_list())
        prediction_df = no_string_match.assign(lstm_prediction = lstm_predictions[:,1])
        print("LSTM predictions done")
        
        # filter out all samples above the threshold
        might_have_ds = prediction_df[prediction_df.lstm_prediction > self.threshold]
        no_ds = prediction_df[~prediction_df.index.isin(might_have_ds)]
        no_ds = no_ds.assign(PredictionString = "")
               
        # get Scibert extraction
        print("Start with the (Sci)bert extraction")
        # for all sentences which do not have direct string matches, use BERT
        dataset_string = self.extract_dataset(might_have_ds)
        print("(Sci)bert extraction done")
        # join both dfs together
        final_df = pd.concat([string_match,no_ds, dataset_string])
        return final_df
        
    def extract_dataset(self,df):
        if len(df) > self.processing_batch * 2:
            batches = np.split(df, int(len(df))/self.processing_batch)
        else:
            batches = np.split(df, 2)

        success = 0
        fail = 0
        result_dfs = []
        
        for batch_df in batches:
            try:
                extracted_datasets = self.bert.predict(batch_df)    
                result_dfs.append(batch_df.assign(PredictionString = extracted_datasets))
                success +=1
            except:
                fail += 1
                result_dfs.append(batch_df.assign(PredictionString = ['failed when retrieving bert']*len(batch_df)))   
            print(f"Bert extraction succeeded in {success} of the cases and failed in {fail}")
        return pd.concat(result_dfs)
        
    def add_dictionary(self,list_of_dataset):
        """
        create the dictionary which contains all datasets from the training data
        """
        clean_ds = set([custom_cleaning(ds) for ds in list_of_dataset if not ds == ""])
        self.dictionary = list(clean_ds)

    def lstm_prediction(self, text):
        """
        obtain the predictions from the lstm model
        """
        padded_x, tokenizer = self.tokenize(text)
        tokens = tokenizer.texts_to_sequences(text)
        predictions = self.lstm_model.predict(padded_x, batch_size = 32)
        return predictions
        
    def load_lstm(self,path):
        """
        loads the LSTM model into the pipeline
        """
        model = keras.models.load_model(path)
        return model
        
    
    def tokenize(self, text, n_words=13615, sequence_length=140): 
        """
        tokenizes the text for the lstm model
        """
        tokenizer = Tokenizer(num_words=n_words, filters=r'!"#$%&()*+,-.:;<=>?@[\]^_`{|}~', lower=True)
        tokenizer.fit_on_texts(text)   
        X = tokenizer.texts_to_sequences(text)
        X = pad_sequences(X, maxlen=sequence_length, padding='pre')
        return X, tokenizer
    
    def bert_prediction(self, df):
        pass
    
    def string_matching(self, df):
        results = df.clean_text.apply(lambda x: find_match(x, self.dictionary))
        df = df.assign(PredictionString=results)
        return df
        
def find_match(sentence:str, dictionary:list):
    """
    Literal string match
    """
    try:
        matches = [ds for ds in dictionary if boyer_moore_match(sentence, ds) != -1]
        if len(matches) > 0:
            # return the longest
            return sorted(matches, key=len, reverse=True)[0]
        else:
            return pd.NA
    except: 
        return f"failed in Find match for sample: {sentence}"

    
pipeline = Pipeline("../input/lstm-for-ds/", "../input/coleridge-bert-models/output", THRESHOLD, BATCH, gpu=True)
pipeline.add_dictionary([custom_cleaning(label) for label in train_df.dataset_label.unique()])

## Execution

In [ ]:
if SUBMISSION:
    result = pipeline.predict(submission)
else:
    result = pipeline.predict(eval_df)
    
result.head()

Print a list of all predicted datasets

In [ ]:
result.PredictionString.unique()

In [ ]:
len(result.PredictionString.unique())

In [ ]:
predictions_list = []
pub_id_list = []

for pub_id, df in result.groupby("pub_id"):
    predictions = df.PredictionString.unique()
    predictions = [p for p in predictions if p != "" and not isinstance(p, float)]
    predicted_datasets = "|".join(predictions) if len(predictions) > 0 else " "
    predictions_list.append(predicted_datasets)
    pub_id_list.append(pub_id)

In [ ]:
submission = pd.DataFrame()
submission['Id'] = pub_id_list
submission['PredictionString'] = predictions_list

submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
result.to_pickle("result.pkl")

In [ ]:
!rm -r predict

## Evaluation

In [ ]:
class Score:
    def __init__(self, output, gt_col, pred_col):
        
        self.final_score = self.score_df_coleridge_initiative(output, gt_col, pred_col, beta=0.5, verbose=True) 
        #lambda output,gt_col,pred_col : self.score_df_coleridge_initiative(output, gt_col, pred_col, beta=0.5, verbose=True)  
    
    def score_df_coleridge_initiative(self,output, gt_col, pred_col, beta=0.5, verbose=True):
        output['evaluation'] = output.apply(lambda x: self.coleridge_initiative_jaccard(x[gt_col], x[pred_col], verbose=False), axis=1)
        output['js_scores'] = output['evaluation'].apply(lambda x : x[0])
        output['pred_type'] = output['evaluation'].apply(lambda x : x[1])
        output['tp_fp_fn'] = output['pred_type'].apply(lambda x : self.get_count_tp_fp_fn(x, verbose=False))
        output = self.make_col_tp_fp_fn(output, 'tp_fp_fn')
        tp = sum(output['TP'])
        fp = sum(output['FP'])
        fn = sum(output['FN'])
        precision, recall = self.get_precision_recall(tp, fp, fn)
        fbeta = self.fbeta_score(precision, recall, 0.5)
        if verbose:
            print("True Positives (TP) : ", tp)
            print("False Positives (FP) : ", fp)
            print("False Negatives (FN) : ", fn)
            print("Precision : ", precision)
            print("Recall : ", recall)
            print("FBeta Score : ", fbeta)
            display(output.head())
        return fbeta
    
    def jaccard(self,str1, str2): 
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        
        # check for 0 in division here
        if (len(a) + len(b) - len(c)) > 0:
            return float(len(c)) / (len(a) + len(b) - len(c))
        else:
            return 0

    def get_precision_recall(self,tp, fp, fn):
        precision = tp / (tp+fp)
        recall = tp / (tp + fn)
        return precision, recall

    def fbeta_score(self,precision, recall, beta):
        fbeta = (1+(beta*beta))*((precision*recall)/( (beta*beta*precision) + recall))
        return fbeta

    def coleridge_initiative_jaccard(self,ground_truth, prediction, verbose=True):
        gts = ground_truth.split('|')
        pds = sorted(prediction.split('|'))
        if verbose:
            print("Ground truth : " , gts)
            print("Prediction : ", pds)
        js_scores = []
        cf_matrix = []
        for pd in pds:
            score = -1
            for gt in gts:
                js = self.jaccard(pd, gt)
                if js > score:
                    score = js
            if score >= 0.5:
                js_scores.append(score)
                cf_matrix.append("TP")
            else:
                js_scores.append(score)
                cf_matrix.append("FP")
        for gt in gts:
            score = -1
            for pd in pds:
                js = self.jaccard(gt, pd)
                if js > score:
                    score = js
            if score == 0:
                js_scores.append(score)
                cf_matrix.append("FN")     
        return js_scores, " ".join(cf_matrix)

    def get_count_tp_fp_fn(self,prediction, verbose=True):
        preds = prediction.split(" ")
        if verbose:
            print(preds)
        tpc = 0
        fpc = 0
        fnc = 0
        for pred in preds:
            if pred == "TP":
                tpc = tpc + 1
            elif pred == "FP":
                fpc = fpc + 1
            elif pred == "FN":
                fnc = fnc + 1
        return [tpc, fpc, fnc]

    def make_col_tp_fp_fn(self,df, col):
        df['TP'] = df[col].apply(lambda x : x[0])
        df['FP'] = df[col].apply(lambda x : x[1])
        df['FN'] = df[col].apply(lambda x : x[2])
        return df

In [ ]:
if SUBMISSION:
    print("no evaluation possible")
else:
    scoring_result = Score(result, "label", "PredictionString")
    print(scoring_result.final_score)